<a href="https://colab.research.google.com/github/skorpiom/traffic_sign_rec/blob/master/traffic_sign_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf sample_data
!rm -rf train_and_valid_signs

In [0]:
%tensorflow_version 2.x
import os
import numpy as np
import pandas as pd
from pandas.io.parsers import read_csv
import matplotlib.pyplot as plt
import shutil
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import VGG19

np.set_printoptions(precision = 6, suppress = True)

In [0]:
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB-Training_fixed.zip
!unzip -q GTSRB-Training_fixed.zip

In [0]:
base_dir = './GTSRB/Training'
raw_no_of_files = {}
classes = []
for n in range(0,43):
  if n<10:
    classes.append("0000"+str(n))
  else:
    classes.append("000"+str(n))

for dir in classes:
  raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir,dir)))

raw_no_of_files.items()

In [0]:
data_dir = './train_and_valid_signs'
dirs_train = []
dirs_valid = []

train_dir = os.path.join(data_dir,'train') #katalog treningowy
valid_dir = os.path.join(data_dir,'valid') #katalog walidacyjny

for n in range(0,43):
  dirs_train.append(os.path.join(train_dir,classes[n]))

for n in range(0,43):
  dirs_valid.append(os.path.join(train_dir,classes[n]))

if not os.path.exists(data_dir):
  os.mkdir(data_dir)
  
for directory in (train_dir,valid_dir):
  if not os.path.exists(directory):
    os.mkdir(directory)

for directory in dirs_train:
  if not os.path.exists(directory):
    os.mkdir(directory)

for directory in dirs_valid:
  if not os.path.exists(directory):
    os.mkdir(directory)

#DISPLAY INDICATED SIGN

In [0]:
img_path = os.path.join('./GTSRB','Training','00000','00001_00010.ppm')
print(img_path)
img = image.load_img(img_path)
plt.figure(figsize=(1,1))
plt.imshow(img)
plt.grid(False)
plt.axis(False)

In [0]:
sign_fnames = []
for ind in range(0,43):
  sign_fnames.append(os.listdir(os.path.join(base_dir,classes[ind])))  

#print(len(sign_fnames[0]))